In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from ase import units
from scipy.integrate import trapezoid

### Frenkel–Ladd path

In [2]:
def integrate_switching(
    df_log: pd.DataFrame,
    equil_time: int = 20000,
    switch_time: int = 30000,
    return_E_diss: bool = False,
):
    fwd_start, fwd_end = equil_time, equil_time + switch_time
    rev_start, rev_end = 2 * equil_time + switch_time, 2 * equil_time + 2 * switch_time
    grad, lamda = df_log["lambda_grad"], df_log["lambda"]
    W_fwd = trapezoid(grad[fwd_start:fwd_end], lamda[fwd_start:fwd_end])
    W_rev = trapezoid(grad[rev_start:rev_end], lamda[rev_start:rev_end])
    if return_E_diss:
        return (W_fwd - W_rev) / 2, (W_fwd + W_rev) / 2
    return (W_fwd - W_rev) / 2  # free energy difference


def analyze_frenkel_ladd(
    base_path: Path,
    temp: float,
    equil_time: int = 20000,
    switch_time: int = 30000,
):
    T = temp
    df_log = pd.read_csv(base_path / "observables.csv")
    k = np.load(base_path / "spring_constants.npy")
    mass = np.load(base_path / "masses.npy")
    omega = np.sqrt(k / mass)
    volume = df_log["volume"].values[0]

    delta_F = integrate_switching(df_log, equil_time, switch_time)
    F_E = 3 * units.kB * T * np.mean(np.log(units._hbar * omega / (units.kB * T)))
    PV = volume * 1.01325 * units.bar
    delta_G = delta_F + F_E + PV

    return delta_G


def analyze_alchemical_switching(
    base_path: Path,
    temp: float,
    equil_time: int = 20000,
    switch_time: int = 30000,
):
    T = temp
    df_log = pd.read_csv(base_path / "observables.csv")
    mass_init = np.load(base_path / "masses_init.npy")
    mass_final = np.load(base_path / "masses_final.npy")

    work = integrate_switching(df_log, equil_time, switch_time)
    G_mass = 1.5 * units.kB * T * np.mean(np.log(mass_init / mass_final))
    delta_G = work + G_mass

    return delta_G

In [3]:
result_path = Path("../data/results/perovskite/frenkel_ladd")
temp_range = [300, 350, 400, 450, 500]

G_alpha = []
G_alpha_std = []
for temp in temp_range:
    G_list = []
    for i in range(4):
        base_path = result_path / f"CsPbI3_alpha_6x6x6_{temp}K/{i}"
        G_list.append(analyze_frenkel_ladd(base_path, temp=temp))
    G = np.mean(G_list)
    G_std = np.std(G_list)
    print(f"CsPbI3 Alpha G ({temp} K) = {G:.4f} ± {G_std:.4f} eV/atom")
    G_alpha.append(G)
    G_alpha_std.append(G_std)

G_delta = []
G_delta_std = []
for temp in temp_range:
    G_list = []
    for i in range(4):
        base_path = result_path / f"CsPbI3_delta_6x3x3_{temp}K/{i}"
        G_list.append(analyze_frenkel_ladd(base_path, temp=temp))
    G = np.mean(G_list)
    G_std = np.std(G_list)
    print(f"CsPbI3 Delta G ({temp} K) = {G:.4f} ± {G_std:.4f} eV/atom")
    G_delta.append(G)
    G_delta_std.append(G_std)

CsPbI3 Alpha G (300 K) = -8.8514 ± 0.0003 eV/atom
CsPbI3 Alpha G (350 K) = -9.8642 ± 0.0004 eV/atom
CsPbI3 Alpha G (400 K) = -10.8783 ± 0.0006 eV/atom
CsPbI3 Alpha G (450 K) = -11.8943 ± 0.0003 eV/atom
CsPbI3 Alpha G (500 K) = -12.9119 ± 0.0004 eV/atom
CsPbI3 Delta G (300 K) = -8.8560 ± 0.0001 eV/atom
CsPbI3 Delta G (350 K) = -9.8660 ± 0.0001 eV/atom
CsPbI3 Delta G (400 K) = -10.8782 ± 0.0002 eV/atom
CsPbI3 Delta G (450 K) = -11.8921 ± 0.0002 eV/atom
CsPbI3 Delta G (500 K) = -12.9072 ± 0.0002 eV/atom


In [4]:
G_CsSnI3_alpha = []
G_CsSnI3_alpha_std = []
for temp in temp_range:
    G_list = []
    for i in range(4):
        base_path = result_path / f"CsSnI3_alpha_6x6x6_{temp}K/{i}"
        G_list.append(analyze_frenkel_ladd(base_path, temp=temp))
    G = np.mean(G_list)
    G_std = np.std(G_list)
    print(f"CsSnI3 Alpha G ({temp} K) = {G:.4f} ± {G_std:.4f} eV/atom")
    G_CsSnI3_alpha.append(G)
    G_CsSnI3_alpha_std.append(G_std)

G_CsSnI3_delta = []
G_CsSnI3_delta_std = []
for temp in temp_range:
    G_list = []
    for i in range(4):
        base_path = result_path / f"CsSnI3_delta_6x3x3_{temp}K/{i}"
        G_list.append(analyze_frenkel_ladd(base_path, temp=temp))
    G = np.mean(G_list)
    G_std = np.std(G_list)
    print(f"CsSnI3 Delta G ({temp} K) = {G:.4f} ± {G_std:.4f} eV/atom")
    G_CsSnI3_delta.append(G)
    G_CsSnI3_delta_std.append(G_std)

CsSnI3 Alpha G (300 K) = -8.8297 ± 0.0002 eV/atom
CsSnI3 Alpha G (350 K) = -9.8413 ± 0.0002 eV/atom
CsSnI3 Alpha G (400 K) = -10.8544 ± 0.0002 eV/atom
CsSnI3 Alpha G (450 K) = -11.8695 ± 0.0003 eV/atom
CsSnI3 Alpha G (500 K) = -12.8863 ± 0.0003 eV/atom
CsSnI3 Delta G (300 K) = -8.8289 ± 0.0001 eV/atom
CsSnI3 Delta G (350 K) = -9.8381 ± 0.0000 eV/atom
CsSnI3 Delta G (400 K) = -10.8494 ± 0.0003 eV/atom
CsSnI3 Delta G (450 K) = -11.8627 ± 0.0003 eV/atom
CsSnI3 Delta G (500 K) = -12.8771 ± 0.0003 eV/atom


### Alchemical path

In [5]:
result_path = Path("../data/results/perovskite/alchemy")

G_alpha = []
G_alpha_std = []
for temp in temp_range:
    G_list = []
    for i in range(4):
        base_path = result_path / f"CsPbI3_CsSnI3_alpha_{temp}K/{i}"
        G_list.append(analyze_alchemical_switching(base_path, temp=temp))
    G = np.mean(G_list)
    G_std = np.std(G_list)
    print(f"Alpha ΔG ({temp} K) = {G:.4f} ± {G_std:.4f} eV/atom")
    G_alpha.append(G)
    G_alpha_std.append(G_std)

G_delta = []
G_delta_std = []
for temp in temp_range:
    G_list = []
    for i in range(4):
        base_path = result_path / f"CsPbI3_CsSnI3_delta_{temp}K/{i}"
        G_list.append(analyze_alchemical_switching(base_path, temp=temp))
    G = np.mean(G_list)
    G_std = np.std(G_list)
    print(f"Delta ΔG ({temp} K) = {G:.4f} ± {G_std:.4f} eV/atom")
    G_delta.append(G)
    G_delta_std.append(G_std)

Alpha ΔG (300 K) = 0.0233 ± 0.0001 eV/atom
Alpha ΔG (350 K) = 0.0236 ± 0.0001 eV/atom
Alpha ΔG (400 K) = 0.0241 ± 0.0001 eV/atom
Alpha ΔG (450 K) = 0.0249 ± 0.0000 eV/atom
Alpha ΔG (500 K) = 0.0258 ± 0.0000 eV/atom
Delta ΔG (300 K) = 0.0271 ± 0.0000 eV/atom
Delta ΔG (350 K) = 0.0279 ± 0.0000 eV/atom
Delta ΔG (400 K) = 0.0286 ± 0.0000 eV/atom
Delta ΔG (450 K) = 0.0294 ± 0.0000 eV/atom
Delta ΔG (500 K) = 0.0301 ± 0.0000 eV/atom
